In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S18'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #From 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 10000
Total Tokens Found = 130023
Total Tokens Output = 127367
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 10000
Total Tokens= 127367
Unique Tokens= 20063
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 10000
Sorted Token Size = 20063
Clean Token Size = 7551
*Stop Replacements here
References Processed =  10000
Total Replacement Pairs = 2321
Tokens Read = 127367
Tokens Changed =  2717
References Changed = 2470


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 10202
NewMatrix Iterateblocks Total Time = 11.406887887045741


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9561
NewMatrix Iterateblocks Total Time = 10.460538209415972


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 8200
NewMatrix Iterateblocks Total Time = 9.00571606028825


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5203
NewMatrix Iterateblocks Total Time = 5.492177749052644


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!

0.72553812

Import BO library:

In [8]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [9]:
import os, sys

In [10]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [11]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [17]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_pos,sigma_pos,mu_01,muIncr_01,epsilon_pos,epsilonIterate_01=0):    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=int(beta_pos),
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=int(sigma_pos),
                      mu=mu_01)
    
    return f1

In [16]:
entropy_F1_trx(beta_pos=33,
               sigma_pos=449,
               mu_01=0.5,
               muIncr_01=0.05,
               epsilon_pos=31.8)

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


0.72553812

Bounds for experiment

In [41]:
beta_levels=(1,500,1000)
sigma_levels=(1,500,1000)
mu_levels=(0.5,0.75,0.99)
muIncr_levels=(0.01,0.1,0.2)
epsilon_levels=(1,500,1000)

In [42]:
optimizer=None

In [43]:
pbounds = {'beta_pos': (beta_levels[0], beta_levels[2]), 
           'sigma_pos': (sigma_levels[0], sigma_levels[2]),
           'mu_01': (mu_levels[0], mu_levels[2]),
           'muIncr_01': (muIncr_levels[0], muIncr_levels[2]),
           'epsilon_pos': (epsilon_levels[0], epsilon_levels[2])}#,

In [44]:
from bayes_opt import SequentialDomainReductionTransformer

In [45]:
bounds_transformer = SequentialDomainReductionTransformer()

In [46]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [47]:
import pyDOE2

In [48]:
levels = [3, 3, 3, 3, 3]

In [49]:
gsd_design=pyDOE2.gsd(levels=levels,reduction=5)

In [50]:
for exp in gsd_design:
    optimizer.probe(
        params={"beta_pos": beta_levels[exp[0]], 
                "sigma_pos": sigma_levels[exp[1]],
                "mu_01":mu_levels[exp[2]],
                "muIncr_01":muIncr_levels[exp[3]],
                "epsilon_pos":epsilon_levels[exp[4]]},
        lazy=True,
    )

In [ ]:
optimizer.maximize(
    init_points=0,
    n_iter=100-gsd_design.shape[0],
    acq="ucb",
    kappa=2
)

|   iter    |  target   | beta_pos  | epsilo... | muIncr_01 |   mu_01   | sigma_pos |
-------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.01|Beta:1.00|Sigma:1.00|Epsilon:1.00|EpIter:0.00
|  1        |  0.0      |  1.0      |  1.0      |  0.01     |  0.5      |  1.0      |
Mu:0.50|Mu+:0.10|Beta:1.00|Sigma:1.00|Epsilon:500.00|EpIter:0.00
|  2        |  0.0      |  1.0      |  500.0    |  0.1      |  0.5      |  1.0      |
Mu:0.50|Mu+:0.20|Beta:1.00|Sigma:1.00|Epsilon:1000.00|EpIter:0.00
|  3        |  0.0      |  1.0      |  1e+03    |  0.2      |  0.5      |  1.0      |
Mu:0.75|Mu+:0.01|Beta:1.00|Sigma:1.00|Epsilon:500.00|EpIter:0.00
|  4        |  0.0      |  1.0      |  500.0    |  0.01     |  0.75     |  1.0      |
Mu:0.75|Mu+:0.10|Beta:1.00|Sigma:1.00|Epsilon:1000.00|EpIter:0.00
|  5        |  0.0      |  1.0      |  1e+03    |  0.1      |  0.75     |  1.0      |
Mu:0.99|Mu+:0.01|Beta:1.00|Sigma:1.00|Epsilon:1000.00|EpIter:0.00
|  6   

|  39       |  0.005111 |  1e+03    |  1.0      |  0.1      |  0.5      |  1e+03    |
Mu:0.50|Mu+:0.20|Beta:1000.00|Sigma:1000.00|Epsilon:500.00|EpIter:0.00
|  40       |  0.002291 |  1e+03    |  500.0    |  0.2      |  0.5      |  1e+03    |
Mu:0.75|Mu+:0.01|Beta:1000.00|Sigma:1000.00|Epsilon:1.00|EpIter:0.00
|  41       |  0.008143 |  1e+03    |  1.0      |  0.01     |  0.75     |  1e+03    |
Mu:0.75|Mu+:0.10|Beta:1000.00|Sigma:1000.00|Epsilon:500.00|EpIter:0.00
|  42       |  0.5791   |  1e+03    |  500.0    |  0.1      |  0.75     |  1e+03    |
Mu:0.75|Mu+:0.20|Beta:1000.00|Sigma:1000.00|Epsilon:1000.00|EpIter:0.00
|  43       |  0.5791   |  1e+03    |  1e+03    |  0.2      |  0.75     |  1e+03    |
Mu:0.99|Mu+:0.01|Beta:1000.00|Sigma:1000.00|Epsilon:500.00|EpIter:0.00
|  44       |  0.02905  |  1e+03    |  500.0    |  0.01     |  0.99     |  1e+03    |
Mu:0.99|Mu+:0.10|Beta:1000.00|Sigma:1000.00|Epsilon:1000.00|EpIter:0.00
|  45       |  0.02905  |  1e+03    |  1e+03    |  0.1    

In [32]:
optimizer.max

{'target': 0.57320787,
 'params': {'beta_pos': 350.6142455450391,
  'epsilon_pos': 28.601690735697275,
  'muIncr_01': 0.1037212333494025,
  'mu_01': 0.7275545713438822,
  'sigma_pos': 699.3887764624509}}

In [34]:
optimizer.max["target"]

0.57320787

In [35]:
params=optimizer.max["params"]

In [36]:
params["beta_pos"]

350.6142455450391

In [37]:
params["sigma_pos"]

699.3887764624509

In [38]:
params["mu_01"]

0.7275545713438822

In [39]:
params["muIncr_01"]

0.1037212333494025

In [40]:
params["epsilon_pos"]

28.601690735697275

In [57]:
entropy_F1(mu=0.5,
           muIncr=0.051004766758926935,
           beta=29,
           sigma=453,
           epsilon=29.18200686707413,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9900
NewMatrix Iterateblocks Total Time = 10.462520653381944


>>>>>>>>>>>>
Starting Iteration mu= 0.551004766758927
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 9174
NewMatrix Iterateblocks Total Time = 10.022749735973775


>>>>>>>>>>>>
Starting Iteration mu= 0.6020095335178539
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 7954
NewMatrix Iterateblocks Total Time = 8.64361101295799


>>>>>>>>>>>>
Starting Iteration mu= 0.6530143002767809
Starting Iteration epsilon= 29.18200686707413
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 5754
NewMatrix Iterateblocks Total Time = 6.570374641567469


>>>>>>>>>>>>
Starting Iteration mu= 0.70401906

0.73010211